In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from NLPcleansing import prep
#from customRNNgraph import RNN_build_graph
from customRNNgraphBidrection import U

In [2]:
MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 18
FORWARD_STEPS = 1
BATCH_SIZE = 32
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_epochs = 20
num_hidden = 100

In [3]:
# read data from twitter
tweet_data   = pd.read_csv('datasets/us_airline.csv')
columns = ['text', 'airline_sentiment']
tweet_data_extract = tweet_data[:][columns]
tweet_data_extract.dropna()
tweet_data_extract["clean_text"] = tweet_data_extract["text"].map(lambda x: prep.text_to_wordlist(x))
tweet_data_extract["labels"] = tweet_data_extract["airline_sentiment"].map(lambda x: prep.sentiment_to_label(x, ['neutral', 'positive', 'negative']))
print('Found %s tweets' % len(tweet_data_extract["clean_text"]))

Found 14640 tweets


In [4]:
# tokenize tweets
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(tweet_data_extract["clean_text"])
sequence_tweets = tokenizer.texts_to_sequences(tweet_data_extract["clean_text"])
word_index = tokenizer.word_index
print('Found %s tokens' % len(word_index))

Found 15074 tokens


In [5]:
if os.path.isfile('embedding_matrix.npy'):
    embedding_matrix = np.load("embedding_matrix.npy")
else:
    GloVe_file = "datasets/glove.6B/glove.6B.300d.txt"
    word2vec_file = "datasets/glove_word2vec.txt"
    embedding_matrix = prep.word2vec_GloVe(GloVe_file, word2vec_file, word_index=word_index)
    np.save("embedding_matrix.npy", embedding_matrix)
nb_words = embedding_matrix.shape[0]

In [6]:
# make sequence to sequence pairs
sequence_tweets_filtered = []
for i in range(len(sequence_tweets)):
    if len(sequence_tweets[i]) >= (MAX_SEQUENCE_LENGTH + FORWARD_STEPS):
        sequence_tweets_filtered.append(sequence_tweets[i])
#data = pad_sequences(sequence_tweets_filtered, maxlen=(MAX_SEQUENCE_LENGTH + FORWARD_STEPS))
#print('Shape of data tensor:', data.shape)

In [7]:
train_seq = []
label_seq = []
for i in range(len(sequence_tweets_filtered)):
    for j in range((len(sequence_tweets_filtered[i])-MAX_SEQUENCE_LENGTH-FORWARD_STEPS+1)):
        train_seq.append(sequence_tweets_filtered[i][j:(j+MAX_SEQUENCE_LENGTH)])
        label_seq.append(sequence_tweets_filtered[i][(j+FORWARD_STEPS):(j+MAX_SEQUENCE_LENGTH+FORWARD_STEPS)])
np_train_seq = np.array(list(train_seq), dtype='int32')
np_label_seq = np.array(list(label_seq), dtype='int32')

########################################
## sample train/validation data
########################################
np.random.seed(1234)
perm = np.random.permutation(len(np_train_seq))
idx_train = perm[:int(len(np_train_seq)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(np_train_seq)*(1-VALIDATION_SPLIT)):]

data_train = np_train_seq[idx_train]
data_val = np_train_seq[idx_val]

labels_train = np_label_seq[idx_train]
labels_val = np_label_seq[idx_val]

print('dtype of train data: %s' % data_train.dtype)
print('dtype of train label: %s' % labels_train.dtype)

print('shape of train data: ', data_train.shape)
print('shape of train label: ', labels_train.shape)

steps_per_train_epoch = int(data_train.shape[0]/BATCH_SIZE)+1
steps_per_valid_epoch = int(data_val.shape[0]/BATCH_SIZE)+1
print('TRIAN STEP %d' % steps_per_train_epoch)
print('VALID STEP %d' % steps_per_valid_epoch)

data = {'train_data': data_train, 'train_labels': labels_train, 'embedding': embedding_matrix, 
        'test_data': data_val, 'test_labels': labels_val}

dtype of train data: int32
dtype of train label: int32
shape of train data:  (40626, 18)
shape of train label:  (40626, 18)
TRIAN STEP 1270
VALID STEP 142


In [8]:
g_bidirect = RNN_bidirect_build_graph(state_size = EMBEDDING_DIM,
                                      num_words = nb_words, 
                                      num_classes = 3, 
                                      batch_size = BATCH_SIZE,
                                      sequence_length = MAX_SEQUENCE_LENGTH,
                                      cell_type = 'LN_LSTM',
                                      num_layers = 3,
                                      init_trainable = False)
t = time.time()
save = "models/bidirect_lstm/base_bidirect/LN_LSTM_bidirect_word_prediction.ckpt"
losses = g_bidirect.train_base_model(data, 8, learning_rate = 1e-4, save=save, verbose_graph=False)
print("It took", time.time() - t, "seconds to train for 100 epochs.")
print("The average loss on the final epoch was:", losses['training_losses'][-1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
training start ...
Average training loss for Epoch 0 : 5.501360336438878
Average  testing loss for Epoch 0 : 3.672854577991324
Average training loss for Epoch 1 : 2.7771549058711433
Average  testing loss for Epoch 1 : 2.092655859362911
Average training loss for Epoch 2 : 1.6654240530306899
Average  testing loss for Epoch 2 : 1.3724791415980164
Average training loss for Epoch 3 : 1.1080113868075092
Average 